In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [ ]:
name_slug_list = []

url = 'https://www.coffeereview.com/advanced-search/page/'

print('begin scraping')
for page in range(1,115,1):
    res = requests.get(url + str(page) + '/',headers = {'User-agent': 'Hanif Al Irsyad'})
    soup = BeautifulSoup(res.content, "lxml")
    names = soup.find_all('p',{'class':"review-title"})
    
    for name in names:
        ns_dict ={}
        ns_dict['name'] = name.text
        ns_dict['slug'] = name.find('a').attrs['href']
        
        name_slug_list.append(ns_dict)
    
    if (page%25) == 0:
        print(f'scraped {page} pages')

    time.sleep(1)

print('done scraping')

begin scraping
done scraping


In [ ]:
name_slug_df = pd.DataFrame(name_slug_list)

In [ ]:
name_slug_df.head()

,name,slug
0,Guatemala El Injerto Pacamara,https://www.coffeereview.com/review/guatemala-...
1,Honduras Roberto Figueroa,https://www.coffeereview.com/review/honduras-r...
2,Guatemala Tojquia,https://www.coffeereview.com/review/guatemala-...
3,Guatemala Don Angel,https://www.coffeereview.com/review/guatemala-...
4,El Salvador La Lagunita,https://www.coffeereview.com/review/el-salvado...


In [ ]:
name_slug_df.to_csv('slug1.csv', index=False)

In [ ]:
coffee_list = []
i = 0
print('begin scraping')

for slug in name_slug_df.slug:
    i += 1
    url = format(slug)
    res = requests.get(url,headers = {'User-agent': 'Hanif Al Irsyad'})
    soup = BeautifulSoup(res.content, "lxml")
    soup = soup.find('div',{'class':"entry-content"})

    coffee_dict={}

    coffee_dict['slug'] = slug
    coffee_dict['all_text'] = soup.get_text()
    coffee_dict['rating'] = soup.find('span',{'class':"review-template-rating"}).text
    coffee_dict['roaster'] = soup.find('p',{'class':"review-roaster"}).text
    coffee_dict['name'] = soup.find('h1',{'class':"review-title"}).text

    coffee_list.append(coffee_dict)
    
    if (i%500) == 0:
        print(f'scraped {i} pages')
        
    time.sleep(1)
    
print('done scraping')

begin scraping
scraped 500 pages
scraped 1000 pages
scraped 1500 pages
scraped 2000 pages
done scraping


In [ ]:
coffee_df = pd.DataFrame(coffee_list)
coffee_df.head()

,slug,all_text,rating,roaster,name
0,https://www.coffeereview.com/review/guatemala-...,\n\n\n94\n\n\nEquator Coffees\nGuatemala El I...,94,Equator Coffees,Guatemala El Injerto Pacamara
1,https://www.coffeereview.com/review/honduras-r...,\n\n\n93\n\n\nFlower Child Coffee\nHonduras R...,93,Flower Child Coffee,Honduras Roberto Figueroa
2,https://www.coffeereview.com/review/guatemala-...,\n\n\n93\n\n\nWonderstate Coffee\nGuatemala T...,93,Wonderstate Coffee,Guatemala Tojquia
3,https://www.coffeereview.com/review/guatemala-...,\n\n\n92\n\n\nBird Rock Coffee Roasters\nGuat...,92,Bird Rock Coffee Roasters,Guatemala Don Angel
4,https://www.coffeereview.com/review/el-salvado...,\n\n\n92\n\n\nChromatic Coffee\nEl Salvador L...,92,Chromatic Coffee,El Salvador La Lagunita


In [ ]:
coffee_df.to_csv('coffee_df_test.csv', index=False)

In [ ]:
info_list = []

for all_text in coffee_df.index:
    text = coffee_df.loc[all_text].all_text
    data_info = [
            re.findall('Roaster Location: ?\n?(.+)',text),
            re.findall('Coffee Origin: ?\n?(.+)',text),
            re.findall('Roast Level: ?\n?(.+)',text),
            re.findall('Est. Price: ?\n?(.+)',text),
            re.findall('Review Date: ?\n?(.+)',text),
            re.findall('Agtron: ?\n?(.+)',text),
            re.findall('Aroma: ?\n?(\d*\.*\d*)',text),
            re.findall('Acid.+: (\d*\.*\d*)',text),
            re.findall('Body: ?\n?(\d*\.*\d*)',text),
            re.findall('Flavor: ?\n?(\d*\.*\d*)',text),
            re.findall('Aftertaste: ?\n?(\d*\.*\d*)',text),
            re.findall('With Milk: ?\n?(\d*\.*\d*)',text),
            re.findall('Blind Assessment: ?\n?(.+)',text),
            re.findall('Notes: ?\n?(.+)',text),
            re.findall('Bottom Line: ?\n?(.+)\n',text),
    ]
    
    info_list.append(data_info)
    
info_df = pd.DataFrame(info_list,columns = ['location', 'origin', 'roast', 'est_price', 
                                            'review_date', 'agtron', 'aroma', 'acid', 
                                            'body', 'flavor', 'aftertaste', 'with_milk', 
                                            'desc_1', 'desc_2', 'desc_3'])
info_df = info_df.applymap(lambda x: x[0] if x != [] else None)
info_df.head()

,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,"San Rafael, California","La Libertad, Huehuetenango Department, Guatemala",Medium-Light,$22.00/6 ounces,November 2022,58/74,9,9,9,9,8,None,"Complex, multi-layered, resonantly deep. Dark ...",Produced by Arturo Aguirre of El Injerto Farm ...,A fine umami-tending cup with equal impulses t...
1,"Oakland, California","Santa Barbara Department, Honduras",Medium-Light,$25.00/250 grams,November 2022,59/78,9,9,8,9,8,None,"Savory leaning, complexly sweet. Jackfruit, bl...",Produced by Roberto Figueroa entirely of the P...,"A richly nuanced Pacas cup from Honduras, para..."
2,"Viroqua, Wisconsin","Tojquia, Huehuetenango Department, Guatemala",Medium-Light,$21.00/12 ounces,November 2022,60/77,8,9,9,9,8,None,"Vibrant, elegantly layered. Pink grapefruit, p...",Produced by Porfirio Velasquez in partnership ...,"A deeply satisfying, classic washed Guatemala ..."
3,"San Diego, California","La Libertad, Huehuetenango Department, Guatemala",Medium-Light,$19.00/12 ounces,November 2022,61/78,8,9,8,9,8,None,"Sweetly tart, high-toned. Pie cherry, baking c...",Produced by Renardo Ovalle Vides of Finca Don ...,"A bright, juicy Guatemala cup with tart fruit,..."
4,"San Jose, California","Apaneca-Ilamatepec growing region, El Salvador",Medium-Light,$25.99/10 ounces,November 2022,62/78,9,8,8,9,8,None,"Deeply chocolaty, richly sweet. Dark chocolate...",Produced by Gloria Rodriguez of Finca Nejapa (...,"A nuanced, rich El Salvador cup centered aroun..."


In [ ]:
info_df.isnull().sum()

location          1
origin            0
roast             0
est_price         0
review_date       0
agtron            0
aroma            27
acid            333
body              3
flavor            3
aftertaste        3
with_milk      1926
desc_1            0
desc_2            0
desc_3            2
dtype: int64

In [ ]:
df_coffee = pd.concat([coffee_df, info_df], axis =1)
df_coffee.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,https://www.coffeereview.com/review/guatemala-...,\n\n\n94\n\n\nEquator Coffees\nGuatemala El I...,94,Equator Coffees,Guatemala El Injerto Pacamara,"San Rafael, California","La Libertad, Huehuetenango Department, Guatemala",Medium-Light,$22.00/6 ounces,November 2022,58/74,9,9,9,9,8,None,"Complex, multi-layered, resonantly deep. Dark ...",Produced by Arturo Aguirre of El Injerto Farm ...,A fine umami-tending cup with equal impulses t...
1,https://www.coffeereview.com/review/honduras-r...,\n\n\n93\n\n\nFlower Child Coffee\nHonduras R...,93,Flower Child Coffee,Honduras Roberto Figueroa,"Oakland, California","Santa Barbara Department, Honduras",Medium-Light,$25.00/250 grams,November 2022,59/78,9,9,8,9,8,None,"Savory leaning, complexly sweet. Jackfruit, bl...",Produced by Roberto Figueroa entirely of the P...,"A richly nuanced Pacas cup from Honduras, para..."
2,https://www.coffeereview.com/review/guatemala-...,\n\n\n93\n\n\nWonderstate Coffee\nGuatemala T...,93,Wonderstate Coffee,Guatemala Tojquia,"Viroqua, Wisconsin","Tojquia, Huehuetenango Department, Guatemala",Medium-Light,$21.00/12 ounces,November 2022,60/77,8,9,9,9,8,None,"Vibrant, elegantly layered. Pink grapefruit, p...",Produced by Porfirio Velasquez in partnership ...,"A deeply satisfying, classic washed Guatemala ..."
3,https://www.coffeereview.com/review/guatemala-...,\n\n\n92\n\n\nBird Rock Coffee Roasters\nGuat...,92,Bird Rock Coffee Roasters,Guatemala Don Angel,"San Diego, California","La Libertad, Huehuetenango Department, Guatemala",Medium-Light,$19.00/12 ounces,November 2022,61/78,8,9,8,9,8,None,"Sweetly tart, high-toned. Pie cherry, baking c...",Produced by Renardo Ovalle Vides of Finca Don ...,"A bright, juicy Guatemala cup with tart fruit,..."
4,https://www.coffeereview.com/review/el-salvado...,\n\n\n92\n\n\nChromatic Coffee\nEl Salvador L...,92,Chromatic Coffee,El Salvador La Lagunita,"San Jose, California","Apaneca-Ilamatepec growing region, El Salvador",Medium-Light,$25.99/10 ounces,November 2022,62/78,9,8,8,9,8,None,"Deeply chocolaty, richly sweet. Dark chocolate...",Produced by Gloria Rodriguez of Finca Nejapa (...,"A nuanced, rich El Salvador cup centered aroun..."


In [ ]:
df_coffee.to_csv('coffee_fix.csv', index=False)

In [ ]:
df_coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   slug         2280 non-null   object
 1   all_text     2280 non-null   object
 2   rating       2280 non-null   object
 3   roaster      2280 non-null   object
 4   name         2280 non-null   object
 5   location     2279 non-null   object
 6   origin       2280 non-null   object
 7   roast        2280 non-null   object
 8   est_price    2280 non-null   object
 9   review_date  2280 non-null   object
 10  agtron       2280 non-null   object
 11  aroma        2253 non-null   object
 12  acid         1947 non-null   object
 13  body         2277 non-null   object
 14  flavor       2277 non-null   object
 15  aftertaste   2277 non-null   object
 16  with_milk    354 non-null    object
 17  desc_1       2280 non-null   object
 18  desc_2       2280 non-null   object
 19  desc_3       2278 non-null 

In [ ]:
df_coffee.isnull().sum()

slug              0
all_text          0
rating            0
roaster           0
name              0
location          1
origin            0
roast             0
est_price         0
review_date       0
agtron            0
aroma            27
acid            333
body              3
flavor            3
aftertaste        3
with_milk      1926
desc_1            0
desc_2            0
desc_3            2
dtype: int64